ПОПРАВИТЬ ПУТИ

# Выделение маски леса на снимке Sentinel-1 SAR: Обучение моделей

Блокнот разбит на 4 части: обучение сверточной сети с тремя сверточными слоями, обучение ResNet-7, обучение U-Net и обучение Random Forest. 

Оценка качества моделей и формирование выходной маски представлены в соответствующем блокноте.

В качестве предупреждения: быстрее всего происходит обучение U-Net и может занимать в районе 20 минут, далее Random Forest (около 30 минут), дольше всего учатся CNN и ResNet (в зависимости от размера патча, но можно ориентироваться на 1,5 ч).

## Подготовка данных

Загрузка библиотек

In [ ]:
# basic
import os
import joblib
import numpy as np
from osgeo import gdal
from tqdm import tqdm
from tqdm import trange
import random
import matplotlib.pyplot as plt

# preprocessing
from sklearn.preprocessing import StandardScaler

# dl
import torch
from torch import nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision import transforms

# classic ml
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

# metrics
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix, matthews_corrcoef
from sklearn.metrics import balanced_accuracy_score

# seed
torch.manual_seed(42)
random.seed(42)
np.random.seed(42)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.is_available()

Функции, которые будут использоваться во всех моделях. Функции, которые относятся к конкретным моделям, вынесены в соответствующие главы

In [ ]:
# подсчет статистик для каналов изображения
def count_statistics(x):
    x_min = np.min(x)
    x_max = np.max(x)
    x_mean = np.mean(x)
    x_std = np.std(x)
    print(f"min={x_min:.2}  max={x_max:.2}  mean={x_mean:.2}  std={x_std:.2}")

In [ ]:
# расчет метрики Intersection over Union
def IoU(tp, fp, fn):
    return tp / (tp + fp + fn)

### Загрузка и предобработка

Задаем пути на изображение, для которого будет проводиться классификация, и на маску (label)

Открытие изображений, сохранение информации о пространственной привязке для будущего выходного изображения, преобразование в матрицу

In [ ]:
!gdown --id 18RnYJKsWTqHfdYZ1Ej_EY44FpJntrKae # forest_mask
!gdown --id 18jTiMnaGNneKwCLaThF4tEZWXKdT5fF5 # sentinel-1 sar image

In [ ]:
raster_path = "/content/subalos_S1B_20191021_.tif"
mask_path = "/content/forest_mask_.tif"

image = gdal.Open(raster_path, gdal.GA_ReadOnly)

# получаем инфо о пространственной привязке
geo_transform = image.GetGeoTransform()
projection = image.GetProjectionRef()

image_array = image.ReadAsArray()

mask = gdal.Open(mask_path, gdal.GA_ReadOnly)
mask_array = mask.ReadAsArray().astype(np.int8)

image = None 
mask = None

print(image_array.shape)
print(mask_array.shape)

Наш снимок в пикселах имеет размер 6146 на 5008, а также содержит 2 канала: это две поляризации VH и VV.

Нужно нормализовать каналы, поскольку известно, что радиолокационные сигналы достаточно низкие, а сигнал в кросс-поляризации ниже, чем в согласованной (из-за смены поляризации происходит потеря энергии).

In [ ]:
fig, axis = plt.subplots(1, 2, figsize=(15, 7))
axis[0].boxplot([image_array[0].flatten(), image_array[1].flatten()])
axis[1].boxplot([image_array[0].flatten(), image_array[1].flatten()])
axis[1].set_ylim((-0.01, 0.15))
plt.show()

На графике ящиков с усами мы видим, какое влияние оказывает спекл-шум на значения радиолокационных сигналов. Слева представлен график ящиков с усами без ограничений, справа - с приближением к межквартильному размаху. Мы можем видеть, что сигналы в кросс-поляризации действительно ниже, чем в вертикальной согласованной, а также что шумные пикселы существенно превышают в значениях значения, характерные для снимков.

Поскольку речь идет об одном снимке, мы можем нормировать его полностью.

In [ ]:
band_1 = image_array[0]
band_2 = image_array[1]

print('first channel')
count_statistics(band_1)
print()
print('second channel')
count_statistics(band_2)

In [ ]:
# standard scaler для CNN
band_1 = (band_1 - np.mean(band_1)) / np.std(band_1)
band_2 = (band_2 - np.mean(band_2)) / np.std(band_2)

# standard scaler для ResNet, U-Net и Random Forest
band_1 = StandardScaler().fit_transform(band_1)
band_2 = StandardScaler().fit_transform(band_2)

print('first channel')
count_statistics(band_1)
print()
print('second channel')
count_statistics(band_2)

In [ ]:
fig, axis = plt.subplots(1, 2, figsize=(15, 7))
axis[0].boxplot([band_1.flatten(), band_2.flatten()])
axis[1].boxplot([band_1.flatten(), band_2.flatten()])
axis[1].set_ylim((-2.5, 2.5))
plt.show()

In [ ]:
image_norm = np.stack((band_1, band_2), axis=-1)
image_norm.shape

Визуализируем фрагмент изображений: VH-поляризацию, VV-поляризацию и маску леса

In [ ]:
# образцово-показательный кусочек
step = 100
figure1 = image_norm[1000:1000+step, 600:600+step, :1] 
figure2 = image_norm[1000:1000+step, 600:600+step, 1:] 
figure3 = mask_array[1000:1000+step, 600:600+step]

fig, axis = plt.subplots(1, 3)
axis[0].imshow(figure1, cmap='Greys_r')
axis[1].imshow(figure2, cmap='Greys_r')
axis[2].imshow(figure3, cmap='Greys_r')
for a in axis:
    a.axis('off') 

Можно обратить внимание на то, что на снимке не_лес темнее.

Посмотрим на долю лесных пикселов среди всех пикселов в маске:

In [ ]:
print(f'{mask_array[mask_array == 1].size / mask_array.size :.2}')

Доля 63 %, значит, наблюдается небольшой дисбаланс.

## Классификация сверточными сетями



**Функции**, которые будут использоваться для обучения

In [ ]:
# генератор патчей
# так как мы классифицируем центральный пиксел, размер стороны патча должен быть нечетным
class Patcher(Dataset):
    def __init__(self, image, mask, transform, patch_size):
        super().__init__()
              
        assert patch_size % 2, "Нечетные патчи, пожалуйста!"
        self.image = image
        self.mask = mask
        self.transform = transform
        self.patch_size = patch_size
        self.im_h, self.im_w = image.shape[0], image.shape[1]
    
        half_patch = self.patch_size // 2
        # координаты центрального пиксела для восстановления маски
        coord_list = list()
        for central_x in trange(half_patch, self.im_w - half_patch): 
            for central_y in range(half_patch, self.im_h - half_patch):
                # создаем патч, только если он не нулевой
                if (self.image[central_y - half_patch:central_y + half_patch + 1,
                               central_x - half_patch:central_x + half_patch + 1] != 0).all():
                    coord_list.append([central_y, central_x])
        self.coords = np.array(coord_list)
        self.size = len(self.coords)

    def __getitem__(self, indx):
        central_x = self.coords[indx, 1] # на основе координат центрального пиксела
        central_y = self.coords[indx, 0]
        
        half_patch = self.patch_size // 2
        # вырезаем патч
        patch = self.image[central_y - half_patch:central_y + half_patch + 1, 
                           central_x - half_patch:central_x + half_patch + 1]
        
        # определяем класс
        label = self.mask[central_y][central_x]
        return self.transform(patch), torch.tensor(label), indx 
    
    def __len__(self):
        return self.size

In [ ]:
# функция для валидации
def validate(model,
             criterion,
             val_loader):
    cumloss = 0
    loss_history = []
    mcc = []
    with torch.no_grad():
        for batch in val_loader:
            x_train, y_train, coords = batch
            x_train, y_train = x_train.to(device), y_train.to(device)
            y_pred = model(x_train) # get predictions
            loss = criterion(y_pred.squeeze(), y_train.to(torch.float32)) # compute loss
            loss_history.append(loss.cpu().detach().numpy()) # write loss to log
            cumloss += loss
            
            # оценка коэффициента Мэттьюса во время обучения
            y_pred = y_pred.squeeze().cpu()
            y_pred = torch.where(y_pred > 0.5, 1, 0)
            mcc_batch = matthews_corrcoef(y_train.to(torch.float32).cpu(), y_pred)
            mcc.append(mcc_batch)
            
    return cumloss / len(val_loader), loss_history, np.mean(mcc) # mean loss and history

In [ ]:
# функция для обучения
def train(model, train_data, test_data, criterion, optimizer, num_epochs=10):
    loss_hist = []
    val_loss_lst = []
    epochs = trange(num_epochs)
    for epoch in epochs:
        ep_loss = 0
        model.train() # dropout!
        for batch in train_data:
            imgs, labels, coords = batch
            imgs, labels = imgs.to(device), labels.to(device)     
            optimizer.zero_grad()
            outputs = model(imgs)
            loss = criterion(outputs.squeeze(), labels.to(torch.float32))
            loss.backward()
            optimizer.step()
            ep_loss += loss.item()
        loss_hist.append(ep_loss / len(train_data))
        
        model.eval() # dropout!
        val_loss, val_loss_hist, mcc = validate(model, criterion, test_data)
        val_loss_lst.append(val_loss.cpu())
        
        print(f"Epoch={epoch}  loss={loss_hist[epoch]:.4}  val_loss={val_loss:.4}  mcc={mcc:.2}")
    return val_loss_lst, loss_hist

In [ ]:
# функция для валидации обученной модели
# возвращает предсказания и их координаты
# печатает метрики для валидационного набора
def final_validate(model, 
                  criterion,
                  val_loader):
    model.eval()
    cumloss = 0
    labels = []
    outputs = []
    coords = []
    with torch.no_grad():
        for batch in val_loader:
            patch, label, coord = batch
            patch, label = patch.to(device), label.to(device)
            y_pred = model(patch) # get predictions
            loss = criterion(y_pred.squeeze(), label.to(torch.float32)) # compute loss
            cumloss += loss
            y_pred = y_pred.squeeze().cpu()
            y_pred = torch.where(y_pred > 0.5, 1, 0)
            outputs.append(y_pred.numpy())
            labels.append(label.cpu().numpy())
            coords.append(coord)
        
    cumloss = cumloss / len(test_loader)
    outputs = np.concatenate(outputs, axis=0)
    labels = np.concatenate(labels, axis=0)
    coords = np.concatenate(coords, axis=0)
    
    print(f"loss={cumloss:.2}")
    print(f"matthews_correlation_coefficient={matthews_corrcoef(labels, outputs):.2}")
    print(f"ROC_AUC={roc_auc_score(labels, outputs):.2}")
    print(f"Balanced accuracy score={balanced_accuracy_score(labels, outputs):.2}")

    tn, fp, fn, tp = confusion_matrix(labels, outputs).ravel()
    iou = IoU(tp, fp, fn)
    print(f"Intersection over Union = {iou:.2}")
    
    return outputs, coords

Настало время резать на **train-val-test**. Так как снимок содержит порядка 30 млн пикселов, будет иметь достаточно много патчей суммарно (в случае с попиксельной классификацией при помощи сверточной сети - 30 млн патчей), что приводит к существенным временн**ы**м затратам при обучении. По этой причине в исследовании для обучения использовалось только 17 % изображения в качестве тренировочной части, 3 % в качестве валидационной, остальная часть - тестовая. 

In [ ]:
# верхняя граница тренировочных данных
bound_train = 1500
# верхняя граница валидационных данных
bound_val = 2500
# нижняя граница валидационных данных
bound_test = 2700

train_image = image_norm[bound_train:bound_val]
train_labels = mask_array[bound_train:bound_val]

val_image = image_norm[bound_val:bound_test]
val_labels = mask_array[bound_val:bound_test]

print(train_image.shape) # 6146 rows total
print(train_labels.shape)
print(val_image.shape)
print(val_labels.shape)

**Подготовка данных к загрузке в модель**

Зададим трансформации. Мы хотим добавить шум в тренировочные данные, чтобы модель не учила шум в данных, но шума среди аугментаций в Pytorch нет. Добавим его самостоятельно

In [ ]:
class AddGaussianNoise(object):
    def __init__(self, mean=0., std=1.):
        self.std = std
        self.mean = mean
        
    def __call__(self, tensor):
        return tensor + torch.randn(tensor.size()) * self.std + self.mean
    
    def __repr__(self):
        return self.__class__.__name__ + '(mean={0}, std={1})'.format(self.mean, self.std)

In [ ]:
train_transform = transforms.Compose([
    transforms.ToTensor(),
    AddGaussianNoise(0., 0.2)])

valid_transform = transforms.Compose([
    transforms.ToTensor()])

Режем на патчи. Готовим датасет.

In [ ]:
patch_size = 15
train_dataset = Patcher(train_image, train_labels, train_transform, patch_size)
valid_dataset = Patcher(val_image, val_labels, valid_transform, patch_size)

print(len(train_dataset))
print(len(valid_dataset))

In [ ]:
batch_size = 2048

train_loader = DataLoader(dataset=train_dataset,
                          batch_size=batch_size,
                          shuffle=True)

valid_loader = DataLoader(dataset=valid_dataset,
                          batch_size=batch_size,
                          shuffle=False)

### Simple CNN

Собираем модель

In [ ]:
class CNN_s1(nn.Module):
    def __init__(self, patch_size: int = 5):
        super().__init__()
        self.conv_stack = nn.Sequential(
            nn.Conv2d(2, 32, 3, stride=1, padding=1), # shape: [32,patch_size,patch_size]
            nn.ReLU(),
            nn.Conv2d(32, 64, 3, stride=1, padding=1), # shape: [64,patch_size,patch_size] 
            nn.ReLU(),
            nn.Conv2d(64, 128, 3, stride=1, padding=1), # shape: [128,patch_size,patch_size] 
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(128*patch_size*patch_size, 1000),
            nn.Dropout(0.25),
            nn.ReLU(), 
            nn.Linear(1000, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        scores = self.conv_stack(x)
        return scores

Обучение модели

In [ ]:
model_cnn = CNN_s1(patch_size=patch_size).to(device)
optimizer = torch.optim.Adam(model_cnn.parameters(), lr=0.001)
criterion = nn.BCELoss()
val_loss_hist, loss_hist = train(model_cnn, train_loader, valid_loader, criterion, optimizer, num_epochs = 10)

Визуализация изменения лоссов от эпохи к эпохе

In [ ]:
plt.plot(range(10), loss_hist)
plt.plot(range(10), val_loss_hist)
plt.legend(['Train loss', 'Val loss'])
plt.xlabel("Epochs", fontsize=15)
plt.ylabel("Loss", fontsize=15)
plt.show()

Валидация модели на валидационном наборе. Оценка качества модели и сборка результирующего изображения в блокноте, посвященном тестированию.

In [ ]:
_, __ = final_validate(model_cnn, 
                       criterion,
                       valid_loader)

Сохранение модели

In [ ]:
torch.save(model_cnn, "/content/model_cnn.pkl")

### ResNet

Поскольку оригинальные версии ResNet достаточно глубокие и содержат несколько слоев пулинга, они не подходят нам для нашей задачи с маленькими патчами. Тем не менее, архитектуру с пробрасыванием можно попробовать использовать, поэтому была написана мини-версия ResNet с 7 слоями.

In [ ]:
class CustomResnet(nn.Module):
    def __init__(self, class_nums = 1, patch_size=5):
        super(CustomResnet, self).__init__()
        self.activation = nn.ReLU()

        resnet_module = nn.Sequential(
            nn.Conv2d(2, 64, 3, stride=1, padding=1),
            self.activation,
            BasicBlock(64, 64, 1),
            BasicBlock(64, 128, 2),
            BasicBlock(128, 128, 1),
            nn.AdaptiveAvgPool2d((1,1))
        )

        dummy_imput = torch.rand(1, 2, patch_size, patch_size)
        out = resnet_module(dummy_imput)

        self.resnet = resnet_module
        self.fc = nn.Sequential(
            nn.Linear(out.shape[1], class_nums),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.resnet(x)
        x = nn.Flatten()(x)
        scores = self.fc(x)
        return scores

class BasicBlock(nn.Module):
    def __init__(self, conv_in, conv_out, stride_first, activation = nn.ReLU):
        super(BasicBlock, self).__init__()
        self.activation = activation()

        if stride_first == 2:
            downs_module = nn.Sequential(
            nn.Conv2d(conv_in, conv_out, 1, stride=2),
            nn.BatchNorm2d(conv_out)
            )
            self.downsample = downs_module
        else:
            self.downsample = None

        bb_module = nn.Sequential(
            nn.Conv2d(conv_in, conv_out, 3, stride=stride_first, padding=1),
            nn.BatchNorm2d(conv_out),
            self.activation,
            nn.Conv2d(conv_out, conv_out, 3, stride=1, padding=1),
            nn.BatchNorm2d(conv_out)
        )
        self.bb = bb_module

    def forward(self, x):
        x_identity = x
        out = self.bb(x)

        if self.downsample is not None: 
            x_identity = self.downsample(x) 

        out += x_identity
        out = self.activation(out)

        return out

Обучение модели

In [ ]:
model_resnet = CustomResnet(patch_size=patch_size).to(device)
optimizer = torch.optim.Adam(model_resnet.parameters(), lr=0.001)
criterion = nn.BCELoss()
val_loss_hist, loss_hist = train(model_resnet, train_loader, valid_loader, criterion, optimizer, num_epochs = 10)

Визуализация изменения лоссов от эпохи к эпохе

In [ ]:
plt.plot(range(10), loss_hist)
plt.plot(range(10), val_loss_hist)
plt.legend(['Train loss', 'Val loss'])
plt.xlabel("Epochs", fontsize=15)
plt.ylabel("Loss", fontsize=15)
plt.show()

Валидация модели на валидационном наборе

In [ ]:
_, __ = final_validate(model_resnet, 
                        criterion,
                        valid_loader)

Сохранение модели

In [ ]:
torch.save(model_resnet, '/content/model_resnet.pkl')

## Сегментация сверточной сетью U-Net

Поскольку модель сегментирует, а не классифицирует патчи, для нее необходимы отдельные класс для создания патчей и функции для обучения

In [ ]:
# генератор патчей
class Patcher_UNet(Dataset):
    def __init__(self, image, mask, transform, patch_size=256, train_part=True):
        super().__init__()
              
        self.image = image
        self.mask = mask
        self.transform = transform
        self.patch_size = patch_size
        self.im_h, self.im_w = image.shape[0], image.shape[1]
        self.train_part = train_part
    
        coord_list = list()
        # сохраняем координаты верхнего левого угла каждого патча для удобства восстановления маски
        for corner_x in trange(0, self.im_w // self.patch_size * self.patch_size, self.patch_size): 
            for corner_y in range(0,  self.im_h // self.patch_size * self.patch_size, self.patch_size):
                if (self.image[corner_y:corner_y + self.patch_size,
                               corner_x:corner_x + self.patch_size] != 0).all():
                    coord_list.append([corner_y, corner_x])
        if not train_part:
            corner_x = self.im_w - self.patch_size
            for corner_y in range(0,  self.im_h // self.patch_size * self.patch_size, self.patch_size):
                coord_list.append([corner_y, corner_x])
            
        self.coords = np.array(coord_list)
        self.size = len(self.coords)

    def __getitem__(self, indx):
        corner_x = self.coords[indx, 1]
        corner_y = self.coords[indx, 0]
        
        patch = self.image[corner_y:corner_y + self.patch_size, 
                           corner_x:corner_x + self.patch_size]
        label = self.mask[corner_y:corner_y + self.patch_size, 
                           corner_x:corner_x + self.patch_size]
        
        if self.train_part:
            trans = transforms.Compose([transforms.ToTensor()])
            concat = torch.cat((trans(patch), trans(label)))
            concat_transformed = self.transform(concat)
            patch, label = torch.split(concat_transformed, 2)
        else:
            patch, label = self.transform(patch), self.transform(label)
        return patch, label, indx
    
    def __len__(self):
        return self.size

In [ ]:
# функция для валидации
def validate_unet(model,
                  criterion,
                  val_loader):
    cumloss = 0
    loss_history = []
    mcc = []
    with torch.no_grad():
        for batch in val_loader:
            imgs, labels, coords = batch
            imgs, labels = imgs.to(device), labels.to(device)
            y_pred = model(imgs) # get predictions
            loss = criterion(y_pred.squeeze(), labels.squeeze()) # compute loss
            loss_history.append(loss.cpu().detach().numpy()) # write loss to log
            cumloss += loss
            y_pred = y_pred.squeeze().cpu()
            labels = labels.squeeze().cpu()
            y_pred = torch.where(y_pred > 0.5, 1, 0).to(torch.int8)
            mcc_batch = matthews_corrcoef(torch.flatten(labels), torch.flatten(y_pred))
            mcc.append(mcc_batch)
    return cumloss / len(val_loader), loss_history, np.mean(mcc) # mean loss and history

In [ ]:
# функция для обучения
def train_unet(model, train_data, test_data, criterion, optimizer, num_epochs=10):
    loss_hist = []
    val_loss_lst = []
    epochs = trange(num_epochs)
    for epoch in epochs:
        ep_loss = 0
        model.train()
        for batch in train_data:
            imgs, labels, coords = batch
            imgs, labels = imgs.to(device), labels.to(device)     
            optimizer.zero_grad()
            outputs = model(imgs)
            loss = criterion(outputs.squeeze(), labels.squeeze())
            loss.backward()
            optimizer.step()
            ep_loss += loss.item()
        loss_hist.append(ep_loss / len(train_data))
        
        model.eval()
        val_loss, val_loss_hist, mcc = validate_unet(model, criterion, test_data)
        val_loss_lst.append(val_loss.cpu())
        
        if not epoch % 50:
            print(f"Epoch={epoch}  loss={loss_hist[epoch]:.4}  val_loss={val_loss:.4}  mcc={mcc:.2}")
    return val_loss_lst, loss_hist

In [ ]:
class AddGaussianNoise(object):
    def __init__(self, mean=0., std=1.):
        self.std = std
        self.mean = mean
        
    def __call__(self, tensor):
        return tensor + torch.randn(tensor.size()) * self.std + self.mean
    
    def __repr__(self):
        return self.__class__.__name__ + '(mean={0}, std={1})'.format(self.mean, self.std)

Поскольку U-Net классифицирует целые патчи, а не пикселы, необходимо увеличить объем тренировочной части, но незначительно, чтобы модели были сопоставимы. Для удобства сделаем границы кратными размеру патча

In [ ]:
patch_size = 256

bound_train = 1000 
bound_val = bound_train + patch_size*5
bound_test = bound_val + patch_size*3
print(f"bound_val={bound_val}  bound_test={bound_test}")


train_image = image_norm[bound_train:bound_val]
train_labels = mask_array[bound_train:bound_val]

val_image = image_norm[bound_val:bound_test]
val_labels = mask_array[bound_val:bound_test]

print(train_image.shape)
print(val_image.shape)

Так как патчи в случае с сегментацией состоят из множества пикселов, то патчей будет получаться значительно меньше, чем в случае попиксельной классификации сверточными сетями. Поэтому возникает необходимость в увеличении разнообразия аугментации.

Для аугментации, помимо шума, были выбраны случайный поворот, зеркальное отражение и вырезание случайного фрагмента патча.

In [ ]:
# new augmentation
train_transform = transforms.Compose([
    transforms.RandomRotation(degrees=(0, 180)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.RandomCrop(128),
    AddGaussianNoise(0., 0.2)])

valid_transform = transforms.Compose([
    transforms.ToTensor()])

In [ ]:
# new loader
train_dataset = Patcher_UNet(train_image, train_labels, train_transform, patch_size, train_part = True)
valid_dataset = Patcher_UNet(val_image, val_labels, valid_transform, patch_size, train_part = False)
print(len(train_dataset))
print(len(valid_dataset))

In [ ]:
batch_size = 4

train_loader = DataLoader(dataset=train_dataset,
                          batch_size=batch_size,
                          shuffle=True)

valid_loader = DataLoader(dataset=valid_dataset,
                          batch_size=batch_size,
                          shuffle=False)

В качестве функции потерь выбран Binary Dice Loss

In [ ]:
# loss
class BinaryDiceLoss(nn.Module):
    def __init__(self, p=2, epsilon=1e-6):
        super().__init__()
        self.p = p  # pow degree
        self.epsilon = epsilon

    def forward(self, predict, target):
        predict = predict.flatten(1)
        target = target.flatten(1)

        num = torch.sum(torch.mul(predict, target), dim=1) + self.epsilon
        den = torch.sum(predict.pow(self.p) + target.pow(self.p), dim=1) + self.epsilon
        loss = 1 - 2 * num / den

        return loss.mean()  # over batch

Возьмем оригинальную U-Net, но нам вряд ли пригодятся предобученные веса из-за специфики наших данных

In [ ]:
model_unet = torch.hub.load('mateuszbuda/brain-segmentation-pytorch', 'unet',
                       in_channels=2, out_channels=1, init_features=32, pretrained=False)

Обучение. Поскольку U-Net учится значительно быстрее сверточных сетей из первого раздела, количество эпох было значительно увеличено

In [ ]:
# train
model_unet = model_unet.to(device)
optimizer = torch.optim.Adam(model_unet.parameters(), lr=0.001)
criterion = BinaryDiceLoss()
val_loss_hist, loss_hist = train_unet(model_unet, train_loader, valid_loader, criterion, optimizer, num_epochs = 500)

Визуализация изменения лоссов от эпохи к эпохе

In [ ]:
plt.plot(range(500), loss_hist)
plt.plot(range(500), val_loss_hist)
plt.legend(['Train loss', 'Val loss'])
plt.xlabel("Epochs", fontsize=15)
plt.ylabel("Loss", fontsize=15)
plt.show()

Сохранение модели

In [ ]:
torch.save(model_unet, "/content/model_unet.pkl")

## Классификация при помощи Random Forest

Последняя модель - представитель методов классического машинного обучения

Поскольку мы хотим, чтобы тренировочный набор был перемешан, сперва отберем из снимка весь участок для тренировки и валидации, который использовался для сверточных моделей для классификации патчей, и уже из него случайным образом отберем тренировочные и валидационные части.

In [ ]:
bound_train = 1500
bound_val = 2500
bound_test = 2700


train_val_image = image_norm[bound_train:bound_test]
train_val_labels = mask_array[bound_train:bound_test]

In [ ]:
train_val_image = np.reshape(train_val_image, 
                             [train_val_image.shape[0] * train_val_image.shape[1], train_val_image.shape[2]])
train_val_labels = train_val_labels.flatten()
print(train_val_image.shape)
print(train_val_labels.shape)

In [ ]:
train_image_rf, val_image_rf, train_labels_rf, val_labels_rf = train_test_split(
                                                               train_val_image, train_val_labels, 
                                                               test_size=(bound_test - bound_val)/(bound_test - bound_train), 
                                                               random_state=42)

Обучение модели. Количество и глубина деревьев ограничены, чтобы помещаться в памяти.

In [ ]:
rf = RandomForestClassifier(n_estimators=200, max_depth = 25, n_jobs=4, verbose=2)

rf = rf.fit(train_image_rf, train_labels_rf)

y_pred_val = rf.predict(val_image_rf)

Оценка на валидационной части

In [ ]:
print(f"Matthews correlation coefficient={matthews_corrcoef(val_labels_rf, y_pred_val):.2}")
print(f"ROC_AUC={roc_auc_score(val_labels_rf, y_pred_val):.2}")
print(f"Balanced accuracy score={balanced_accuracy_score(val_labels_rf, y_pred_val):.2}")

tn, fp, fn, tp = confusion_matrix(val_labels_rf, y_pred_val).ravel()
iou = IoU(tp, fp, fn)

print(f"Intersection over Union = {iou:.2}")

Сохранение модели

In [ ]:
joblib.dump(rf, "/content/random_forest.joblib") # very heavy model!